In [82]:


import pandas as pd

# Read the Excel file
file_path = r'C:\Crime_prediction\crimes__dataset.csv'

data = pd.read_csv(file_path)

In [83]:
data=data.head(225000)

In [84]:
print(data.info())
print(data.describe())
print(data.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225000 entries, 0 to 224999
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   ID                    225000 non-null  int64  
 1   Case Number           225000 non-null  object 
 2   Date                  225000 non-null  object 
 3   Block                 225000 non-null  object 
 4   IUCR                  225000 non-null  object 
 5   Primary Type          225000 non-null  object 
 6   Description           225000 non-null  object 
 7   Location Description  223298 non-null  object 
 8   Arrest                225000 non-null  bool   
 9   Domestic              225000 non-null  bool   
 10  Beat                  225000 non-null  int64  
 11  District              225000 non-null  float64
 12  Ward                  224830 non-null  float64
 13  Community Area        224835 non-null  float64
 14  FBI Code              225000 non-null  object 
 15  

In [85]:

data.drop(['X Coordinate', 'Y Coordinate','Latitude','Longitude','Beat','FBI Code','Case Number','IUCR','ID','District','Updated On','Block','Domestic','Ward','Community Area','Year','Arrest'], axis=1, inplace=True)


In [86]:
data.dropna(inplace=True)

In [87]:
data.rename(columns={
    'Date': 'date',
    'Primary Type': 'primary_type',
    'Description': 'description',
    'Location Description': 'location_description',
    'Location': 'location'
}, inplace=True)

In [88]:

data['date'] = pd.to_datetime(data['date'])


data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day
data['hour'] = data['date'].dt.hour
data['minute'] = data['date'].dt.minute

C:\Users\nischay\AppData\Local\Temp\ipykernel_13420\1010124532.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['date'] = pd.to_datetime(data['date'])


In [89]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 220480 entries, 1 to 224999
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   date                  220480 non-null  datetime64[ns]
 1   primary_type          220480 non-null  object        
 2   description           220480 non-null  object        
 3   location_description  220480 non-null  object        
 4   location              220480 non-null  object        
 5   year                  220480 non-null  int32         
 6   month                 220480 non-null  int32         
 7   day                   220480 non-null  int32         
 8   hour                  220480 non-null  int32         
 9   minute                220480 non-null  int32         
dtypes: datetime64[ns](1), int32(5), object(4)
memory usage: 14.3+ MB


In [90]:
# Step 1: Remove the leading comma, space, and newline characters
data['location'] = data['location'].str.replace(r'\s*,\s*\n', '', regex=True)

# Step 2: Strip any remaining whitespace
data['location'] = data['location'].str.strip()

# Step 3: Extract x and y coordinates
coordinates = data['location'].str.strip('()').str.split(',', expand=True)

# Check if extraction was successful (i.e., we have two columns)
if coordinates.shape[1] == 2:
    # Step 4: Convert to numeric
    data['x'] = pd.to_numeric(coordinates[0].str.strip(), errors='coerce')
    data['y'] = pd.to_numeric(coordinates[1].str.strip(), errors='coerce')

    # Step 5: Drop rows with missing or invalid coordinates
    data = data.dropna(subset=['x', 'y'])
else:
    print("Error: Could not extract coordinates. Check the format of the 'location' column.")


In [91]:
# Define the target variables (y)
y_hour = data['hour']
y_minute = data['minute']
y_location = data[['x', 'y']]  # Assuming we want to predict both x and y coordinates

# Define the feature variables (X)
X = data.drop(columns=['date', 'hour', 'minute', 'x', 'y', 'location'])

In [92]:
!pip install scikit-learn


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [93]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Define numerical and categorical features
numerical_features = ['year', 'month', 'day']
categorical_features = ['primary_type', 'description', 'location_description']

# Pipeline for numerical features
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Pipeline for categorical features
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine both pipelines into a single preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Apply the transformations to the features
X_preprocessed = preprocessor.fit_transform(X)



In [95]:
from sklearn.model_selection import train_test_split

# Split data for hour prediction
X_train_hour, X_test_hour, y_hour_train, y_hour_test = train_test_split(
    X_preprocessed, y_hour, test_size=0.2, random_state=42
)

# Split data for minute prediction
X_train_minute, X_test_minute, y_minute_train, y_minute_test = train_test_split(
    X_preprocessed, y_minute, test_size=0.25, random_state=42
)

# Split data for location prediction
X_train_location, X_test_location, y_location_train, y_location_test = train_test_split(
    X_preprocessed, y_location, test_size=0.2, random_state=42
)

In [96]:
#TRAINING STEP 8
from sklearn.ensemble import RandomForestRegressor

# Train the model for predicting hour
hour_model = RandomForestRegressor(n_estimators=100, random_state=42,n_jobs=-1)
hour_model.fit(X_train_hour, y_hour_train)

# Train the model for predicting minute
minute_model = RandomForestRegressor(n_estimators=100, random_state=42,n_jobs=-1)
minute_model.fit(X_train_minute, y_minute_train)

# Train the model for predicting x coordinate
x_model = RandomForestRegressor(n_estimators=100, random_state=42,n_jobs=-1)
x_model.fit(X_train_location, y_location_train['x'])

# Train the model for predicting y coordinate
y_model = RandomForestRegressor(n_estimators=100, random_state=42,n_jobs=-1)
y_model.fit(X_train_location, y_location_train['y'])


RandomForestRegressor(n_jobs=-1, random_state=42)

In [97]:
#STEP 9 EVALUATION
from sklearn.metrics import mean_squared_error

# Evaluate hour prediction model
hour_predictions = hour_model.predict(X_test_hour)
hour_mse = mean_squared_error(y_hour_test, hour_predictions)
print(f"Mean Squared Error for Hour Prediction: {hour_mse}")

# Evaluate minute prediction model
minute_predictions = minute_model.predict(X_test_minute)
minute_mse = mean_squared_error(y_minute_test, minute_predictions)
print(f"Mean Squared Error for Minute Prediction: {minute_mse}")

# Evaluate x coordinate prediction model
x_predictions = x_model.predict(X_test_location)
x_mse = mean_squared_error(y_location_test['x'], x_predictions)
print(f"Mean Squared Error for X Coordinate Prediction: {x_mse}")

# Evaluate y coordinate prediction model
y_predictions = y_model.predict(X_test_location)
y_mse = mean_squared_error(y_location_test['y'], y_predictions)
print(f"Mean Squared Error for Y Coordinate Prediction: {y_mse}")


Mean Squared Error for Hour Prediction: 52.282587796703126
Mean Squared Error for Minute Prediction: 382.76354609030545
Mean Squared Error for X Coordinate Prediction: 0.008950534365799984
Mean Squared Error for Y Coordinate Prediction: 0.004224676797962231


In [98]:
from sklearn.metrics import mean_absolute_error

# Calculate Mean Absolute Error
hour_mae = mean_absolute_error(y_hour_test, hour_predictions)
minute_mae = mean_absolute_error(y_minute_test, minute_predictions)
x_mae = mean_absolute_error(y_location_test['x'], x_predictions)
y_mae = mean_absolute_error(y_location_test['y'], y_predictions)

# Convert MAE to percentage
hour_range = y_hour_test.max() - y_hour_test.min()
minute_range = y_minute_test.max() - y_minute_test.min()
x_range = y_location_test['x'].max() - y_location_test['x'].min()
y_range = y_location_test['y'].max() - y_location_test['y'].min()

hour_mae_percentage = (hour_mae / hour_range) * 100
minute_mae_percentage = (minute_mae / minute_range) * 100
x_mae_percentage = (x_mae / x_range) * 100
y_mae_percentage = (y_mae / y_range) * 100

print(f"Mean Absolute Error for Hour Prediction: {hour_mae:.2f} ({hour_mae_percentage:.2f}% of range)")
print(f"Mean Absolute Error for Minute Prediction: {minute_mae:.2f} ({minute_mae_percentage:.2f}% of range)")
print(f"Mean Absolute Error for X Coordinate Prediction: {x_mae:.2f} ({x_mae_percentage:.2f}% of range)")
print(f"Mean Absolute Error for Y Coordinate Prediction: {y_mae:.2f} ({y_mae_percentage:.2f}% of range)")



Mean Absolute Error for Hour Prediction: 5.91 (25.69% of range)
Mean Absolute Error for Minute Prediction: 16.43 (27.84% of range)
Mean Absolute Error for X Coordinate Prediction: 0.07 (1.37% of range)
Mean Absolute Error for Y Coordinate Prediction: 0.05 (1.17% of range)


In [101]:
# Choose an entry after 225000 (for example, the 225001st entry)
single_entry = data.iloc[175000]  # This is the 225001st entry

# Preprocess this entry (assuming the same preprocessing steps)
single_entry_preprocessed = preprocessor.transform(single_entry.to_frame().T)

# Make predictions
predicted_hour = hour_model.predict(single_entry_preprocessed)
predicted_minute = minute_model.predict(single_entry_preprocessed)
predicted_x = x_model.predict(single_entry_preprocessed)
predicted_y = y_model.predict(single_entry_preprocessed)

# Display the predictions
print(f"Predicted Hour: {predicted_hour[0]}")
print(f"Predicted Minute: {predicted_minute[0]}")
print(f"Predicted Location (X Coordinate): {predicted_x[0]}")
print(f"Predicted Location (Y Coordinate): {predicted_y[0]}")


Predicted Hour: 12.246314021037758
Predicted Minute: 32.392693575516326
Predicted Location (X Coordinate): 41.923759685822546
Predicted Location (Y Coordinate): -87.67613025639353


In [105]:
# Choose the entry (e.g., the 225001st entry)
actual_entry = data.iloc[175000]  # This is the 225001st entry

# Display the actual hour, minute, and location values
actual_hour = actual_entry['hour']
actual_minute = actual_entry['minute']
actual_x = actual_entry['location'].split(',')[0].strip().replace('(', '')
actual_y = actual_entry['location'].split(',')[1].strip().replace(')', '')

print(f"Actual Hour: {actual_hour}")
print(f"Actual Minute: {actual_minute}")
print(f"Actual X Coordinate: {actual_x}")
print(f"Actual Y Coordinate: {actual_y}")


Actual Hour: 8
Actual Minute: 35
Actual X Coordinate: 41.943438563
Actual Y Coordinate: -87.644885529
